# Graph I - AdjMatrixGraph#

<img src="./images/ch17/matrixrepr.png" width="640"/>

In [1]:
class Vertex:
    def __init__(self, node):
        self.id = node
        self.visited = False
        
    def addNeighbor(self, neighbor, G):
        G.addEdge(self.id, neighbor)
        
    def getConnections(self, G):
        return G.adjMatrix[self.id]
        
    def getVertexID(self):
        return self.id
    
    def setVertexID(self, id):
        self.id = id

    def setVisited(self):
        self.visited = True
        
    def __str__(self):
        return str(self.id)

class Graph:
    def __init__(self, numVertices=10, directed=False):
        self.adjMatrix = [[None] * numVertices for _ in range(numVertices)]
        self.numVertices = numVertices
        self.directed = directed
        self.vertices = [] # list->dic {id:vertex}
        for i in range(numVertices):
            newVertex = Vertex(i)
            self.vertices.append(newVertex)
        
    def addVertex(self, vtx, id):
        if 0 <= vtx < self.numVertices:
            self.vertices[vtx].setVertexID(id)
      
    def getVertex(self, n):
        for ver_index in range(self.numVertices):
            if self.vertices[ver_index].getVertexID() == n:
                return ver_index
        return None

    def addEdge(self, frm, to, cost=0): 
        if self.getVertex(frm) is not None and self.getVertex(to) is not None:
            self.adjMatrix[self.getVertex(frm)][self.getVertex(to)] = cost
            if not self.directed:
                # For directed graph do not add this
                self.adjMatrix[self.getVertex(to)][self.getVertex(frm)] = cost

    def getVertices(self):
        # *** create a copy, and return a copy ***
        vertices = []
        for v in self.vertices:
            vertices.append(v.getVertexID())
        return vertices
    
    def printMatrix(self):
        for i in range(self.numVertices):
            row = []
            for j in range(self.numVertices):
                row.append(str(self.adjMatrix[i][j]) if self.adjMatrix[i][j] is not None else '/')
            print(row)
        
    def getEdges(self):
        edges = []
        for i in range(self.numVertices):
            for j in range(self.numVertices):
                if self.adjMatrix[i][j] is not None:
                    frm_id = self.vertices[i].getVertexID()
                    to_id = self.vertices[j].getVertexID()
                    edges.append((frm_id,to_id,self.adjMatrix[i][j]))
        return edges
        
    def getNeighbors(self, n):
        neighbors = []
        for ver_index in range(self.numVertices):
            if n == self.vertices[ver_index].getVertexID():
                for neighbor in range(self.numVertices):
                    if self.adjMatrix[ver_index][neighbor] is not None:
                        neighbors.append(self.vertices[neighbor].getVertexID())
        return neighbors
                
    def isConnected(self, u, v):
        uidx = self.getVertex(u) 
        vidx = self.getVertex(v)
        return self.adjMatrix[uidx][vidx] is not None
    
    def get2Hops(self, u):
        neighbors = self.getNeighbors(u)
        hopset = set()
        for v in neighbors:
            hops = self.getNeighbors(v)
            hopset |= set(hops)
        return list(hopset)

In [2]:
graph = Graph(6,True)
graph.addVertex(0, 'a')
graph.addVertex(1, 'b')
graph.addVertex(2, 'c')
graph.addVertex(3, 'd')
graph.addVertex(4, 'e')
graph.addVertex(5, 'f')
graph.addVertex(6, 'g') # doing nothing here 
graph.addVertex(7, 'h') # doing nothing here

print(graph.getVertices())
graph.addEdge('a', 'b', 1)  
graph.addEdge('a', 'c', 2)
graph.addEdge('b', 'd', 3)
graph.addEdge('b', 'e', 4)
graph.addEdge('c', 'd', 5)
graph.addEdge('c', 'e', 6)
graph.addEdge('d', 'e', 7)
graph.addEdge('e', 'a', 8)
print(graph.printMatrix())
print(graph.getEdges())    

['a', 'b', 'c', 'd', 'e', 'f']
['/', '1', '2', '/', '/', '/']
['/', '/', '/', '3', '4', '/']
['/', '/', '/', '5', '6', '/']
['/', '/', '/', '/', '7', '/']
['8', '/', '/', '/', '/', '/']
['/', '/', '/', '/', '/', '/']
None
[('a', 'b', 1), ('a', 'c', 2), ('b', 'd', 3), ('b', 'e', 4), ('c', 'd', 5), ('c', 'e', 6), ('d', 'e', 7), ('e', 'a', 8)]


In [3]:
graph.getNeighbors('a')

['b', 'c']

In [4]:
graph.isConnected('a','e')

False

In [5]:
graph.get2Hops('a')

['d', 'e']

In [6]:
G = Graph(5)
G.addVertex(0, 'a')
G.addVertex(1, 'b')
G.addVertex(2, 'c')
G.addVertex(3, 'd')
G.addVertex(4, 'e')
G.addEdge('a', 'e', 10)  
G.addEdge('a', 'c', 20)
G.addEdge('c', 'b', 30)
G.addEdge('b', 'e', 40)
G.addEdge('e', 'd', 50)
G.addEdge('f', 'e', 60)
print(G.printMatrix())
print(G.getEdges()) 

['/', '/', '20', '/', '10']
['/', '/', '30', '/', '40']
['20', '30', '/', '/', '/']
['/', '/', '/', '/', '50']
['10', '40', '/', '50', '/']
None
[('a', 'c', 20), ('a', 'e', 10), ('b', 'c', 30), ('b', 'e', 40), ('c', 'a', 20), ('c', 'b', 30), ('d', 'e', 50), ('e', 'a', 10), ('e', 'b', 40), ('e', 'd', 50)]


# Graph II - AdjListGraph #

<img src="./images/ch17/listrepr.png" width="640"/>

In [20]:
import sys
class Vertex:
    def __init__(self, node):
        self.id = node
        self.adjacent = {}
        self.visited = False
        self.previous = None
        self.distance = sys.maxsize
        
    def addNeighbor(self, neighbor, weight=0):
        self.adjacent[neighbor] = weight

    # returns a list 
    def getConnections(self): # neighbor keys
        return self.adjacent.keys()

    def getVertexID(self):
        return self.id

    def getWeight(self, neighbor):
        return self.adjacent[neighbor]

    def setDistance(self, dist):
        self.distance = dist
    
    def getDistance(self):
        return self.distance
    
    def setPrevious(self, prev):
        self.previous = prev
        
    def setVisited(self):
        self.visited = True
        
    def __str__(self):
        return str(self.id) + ' adjacent: '+str([x.id for x in self.adjacent])
        
    def __lt__(self, other):
        return self.distance < other.distance and self.id < other.id

class Graph:
    def __init__(self, directed=False):
        # key is string, vertex id
        # value is Vertex
        self.verDictionary = {}
        self.numVertices = 0
        self.directed = directed
        
    def __iter__(self):
        return iter(self.verDictionary.values())
    
    def isDirected(self):
        return self.directed
    
    def vertexCount(self):
        return self.numVertices
    
    def addVertex(self, node):
        self.numVertices += 1
        newVertex = Vertex(node)
        self.verDictionary[node] = newVertex
        return newVertex

    def getVertex(self, n):
        if n in self.verDictionary:
            return self.verDictionary[n]
        else:
            return None

    def addEdge(self, frm, to, cost=0):
        if frm not in self.verDictionary:
            self.addVertex(frm)
        if to not in self.verDictionary:
            self.addVertex(to)
        self.verDictionary[frm].addNeighbor(self.verDictionary[to], cost)
        
        if not self.directed:
            self.verDictionary[to].addNeighbor(self.verDictionary[frm], cost)

    def getVertices(self):
        return self.verDictionary.keys()

    def setPrevious(self, current):
        self.previous = current

    def getPrevious(self, current):
        return self.previous

    def getEdges(self):
        edges = []
        for key, currentVert in self.verDictionary.items():
            currentVertID = currentVert.getVertexID()
            for nbr in currentVert.getConnections():
                nbrID = nbr.getVertexID()
                edges.append((currentVertID, nbrID, currentVert.getWeight(nbr)))
        return edges
                
    def getNeighbors(self, v):
        vertex = self.verDictionary[v]
        return vertex.getConnections()

In [6]:
G = Graph(True)
G.addVertex('a')
G.addVertex('b')
G.addVertex('c')
G.addVertex('d')
G.addVertex('e')
G.addVertex('f')
G.addEdge('a', 'b', 1)  
G.addEdge('a', 'c', 1)
G.addEdge('b', 'd', 1)
G.addEdge('b', 'e', 1)
# G.addEdge('c', 'd', 1)
G.addEdge('c', 'e', 1)
G.addEdge('d', 'e', 1)
G.addEdge('e', 'a', 1)
G.addEdge('a', 'f', 1)
print (G.getEdges())
for k in G.getEdges():
    print(k)

[('a', 'b', 1), ('a', 'c', 1), ('a', 'f', 1), ('b', 'd', 1), ('b', 'e', 1), ('c', 'e', 1), ('d', 'e', 1), ('e', 'a', 1)]
('a', 'b', 1)
('a', 'c', 1)
('a', 'f', 1)
('b', 'd', 1)
('b', 'e', 1)
('c', 'e', 1)
('d', 'e', 1)
('e', 'a', 1)


In [9]:
for key in G.verDictionary:
    print(key, 'corresponds to', G.verDictionary[key])

a corresponds to a adjacent: ['b', 'c', 'f']
b corresponds to b adjacent: ['d', 'e']
c corresponds to c adjacent: ['e']
d corresponds to d adjacent: ['e']
e corresponds to e adjacent: ['a']
f corresponds to f adjacent: []


In [10]:
v = 'a'
neighbors = G.getNeighbors(v)
for n in neighbors:
    print(n)

b adjacent: ['d', 'e']
c adjacent: ['e']
f adjacent: []


In [11]:
def graphFromEdgelist(E, directed=False):
    """Make a graph instance based on a sequence of edge tuples.
    Edges can be either of from (origin,destination) or
    (origin,destination,element). Vertex set is presume to be those
    incident to at least one edge.
    vertex labels are assumed to be hashable.
    """
    g = Graph(directed)
    V = set()
    for e in E:
        V.add(e[0])
        V.add(e[1])
        
    print("Vertex: ", V)
    
    verts = {} # map from vertex label to Vertex instance
    for v in V:
        verts[v] = g.addVertex(v)
    print(g.vertexCount())
    
    for e in E:
        frm = e[0]
        to = e[1]
        cost = e[2] if len(e) > 2 else None
        g.addEdge(frm, to, cost)
    return g

In [12]:
E2 = (
('A','B', 1), ('A','C', 1),
)
graph = graphFromEdgelist(E2, True)
for k in graph.getEdges():
    print(k)

Vertex:  {'B', 'C', 'A'}
3
('A', 'B', 1)
('A', 'C', 1)


In [13]:
E = (
('SFO', 'LAX', 337), ('SFO', 'BOS', 2704), ('SFO', 'ORD', 1846),
('SFO', 'DFW', 1464), ('LAX', 'DFW', 1235), ('LAX', 'MIA', 2342),
('DFW', 'ORD', 802), ('DFW', 'MIA', 1121), ('ORD', 'BOS', 867),
('ORD', 'JFK', 740), ('MIA', 'JFK', 1090), ('MIA', 'BOS', 1258), 
('JFK', 'BOS', 187),
)
graph = graphFromEdgelist(E, True)
for e in graph.getEdges():
    print(e)

for m in graph.getVertices():
    print(m)

Vertex:  {'ORD', 'BOS', 'LAX', 'DFW', 'SFO', 'MIA', 'JFK'}
7
('ORD', 'BOS', 867)
('ORD', 'JFK', 740)
('LAX', 'DFW', 1235)
('LAX', 'MIA', 2342)
('DFW', 'ORD', 802)
('DFW', 'MIA', 1121)
('SFO', 'LAX', 337)
('SFO', 'BOS', 2704)
('SFO', 'ORD', 1846)
('SFO', 'DFW', 1464)
('MIA', 'JFK', 1090)
('MIA', 'BOS', 1258)
('JFK', 'BOS', 187)
ORD
BOS
LAX
DFW
SFO
MIA
JFK


# Graph III DFS #

In [14]:
def dfs_helper(G, Vertex, visited):
    visited.add(Vertex) #标记访问过的节点
    print("traversal: " + Vertex.getVertexID())
    for nbr in Vertex.getConnections():# 列出所有相邻节点，选一个
        if nbr not in visited:# 判断是否访问过
            dfs_helper(G, nbr, visited) #递归调用
    return
    
def DFS_recursion(G):
    visited = set() # 用set记录访问过的结点
    for currentVert in G.getVertices(): #列出图中所有节点，选取一个
        if G.getVertex(currentVert) not in visited: #如果还没访问过，从它开始遍历
            dfs_helper(G, G.getVertex(currentVert),visited) #如果是连通图，这句只执行一次

In [15]:
DFS_recursion(G)

traversal: a
traversal: b
traversal: d
traversal: e
traversal: c
traversal: f


In [16]:
visited = set()
v = G.getVertex('e')
dfs_helper(G, v, visited)

traversal: e
traversal: a
traversal: b
traversal: d
traversal: c
traversal: f


In [25]:
def dfsIterative(G, start, dest):
    stack = []
    visited = set()
    path = {}
    if G.getVertex(start) is not None and G.getVertex(dest) is not None: # 确认起点和终点节点是否存在
        v = G.getVertex(start)
        stack.append(v)
        visited.add(v)
    while len(stack):
        current_vertex = stack.pop()
        print("visiting:",current_vertex.getVertexID())
        if current_vertex.getVertexID() == dest:
            return path
        for nbr in current_vertex.getConnections():
            if nbr not in visited:
                visited.add(nbr)
                path[nbr.getVertexID()] = current_vertex.getVertexID()
                stack.append(nbr)
    return None

In [27]:
parent = dfsIterative(G, 'a', 'd')
print(parent)

visiting: a
visiting: f
visiting: c
visiting: e
visiting: b
visiting: d
{'b': 'a', 'c': 'a', 'f': 'a', 'e': 'c', 'd': 'b'}


# Graph IV BFS #

In [23]:
from collections import deque

def BFS(G, start, dest):
    queue = deque() # 双向队列
    visited = set()
    path = {}
    if G.getVertex(start) is not None and G.getVertex(dest) is not None: # 确认起点和终点节点是否存在
        v = G.getVertex(start)
        queue.append(v)
        visited.add(v)
    while len(queue):
        current_vertex = queue.popleft()
        print("visiting:",current_vertex.getVertexID())
        if current_vertex.getVertexID() == dest:
            return path
        for nbr in current_vertex.getConnections():
            if nbr not in visited:
                visited.add(nbr)
                path[nbr.getVertexID()] = current_vertex.getVertexID()
                queue.append(nbr)
    return None

In [24]:
parent = BFS(G, 'a', 'd')
print(parent)

visiting: a
visiting: b
visiting: c
visiting: f
visiting: d
{'b': 'a', 'c': 'a', 'f': 'a', 'd': 'b', 'e': 'b'}


# Dijkstra Algorithm #

In [25]:
import heapq

def shortest(v,path):
    if v.previous:
        path.append(v.previous.getVertexID())
        shortest(v.previous,path)
    return

def dijkstra(G, source, destination):
    #设置起点的距离是0
    source.setDistance(0)
    
    # 在优先队列（堆）里放入tuple对
    unvisitedQueue = [(v.getDistance(), v) for v in G]
    # 除了起点之外，其它vertex的距离都是无穷大，相当于①堆中只放入了起点vertex
    heapq.heapify(unvisitedQueue)
    
    # 当堆不为空
    while len(unvisitedQueue):
        # python的堆是最小堆，如果放入的是tuple 则以第一个元素为依据排序
        # ② 从堆中pop出距离最小的vertex
        uv = heapq.heappop(unvisitedQueue)
        cur = uv[1]
        cur.setVisited()
        
        # ③ 将邻居放入堆
        for neighbor in cur.adjacent:
            # 如果已经访问过了，则跳过
            if neighbor.visited:
                continue
            newDist = cur.getDistance() + cur.getWeight(neighbor)
            
            # ④ 如果发现了更短的路径，则更新距离
            if newDist < neighbor.getDistance():
                neighbor.setDistance(newDist)
                neighbor.setPrevious(cur)
                print('Updated : current = %s neighbor = %s newDist = %s'\
                     %(cur.getVertexID(), neighbor.getVertexID(), neighbor.getDistance()))
            else:
                print('NOT Updated : current = %s neighbor = %s newDist = %s'\
                     %(cur.getVertexID(), neighbor.getVertexID(), neighbor.getDistance()))
                
        #重构堆
        while len(unvisitedQueue):
            unvisitedQueue.pop()
            
        # 把没访问过的vertex放入堆
        unvisitedQueue = [(v.getDistance(), v) for v in G if not v.visited]
        heapq.heapify(unvisitedQueue)

In [8]:
G = Graph(True)
G.addVertex('a')
G.addVertex('b')
G.addVertex('c')
G.addVertex('d')
G.addVertex('e')
G.addEdge('a', 'b', 4)  
G.addEdge('a', 'c', 1)
G.addEdge('c', 'b', 2)
G.addEdge('b', 'e', 4)
G.addEdge('c', 'd', 4)
G.addEdge('d', 'e', 4)

for v in G:
    for w in v.getConnections():
        vid = v.getVertexID()
        wid = w.getVertexID()
        print('( %s , %s, %3d)' % (vid, wid, v.getWeight(w)))


( a , b,   4)
( a , c,   1)
( b , e,   4)
( c , b,   2)
( c , d,   4)
( d , e,   4)


In [22]:
source = G.getVertex('a')
destination = G.getVertex('e')    
dijkstra(G, source, destination) 

NOT Updated : current = a neighbor = b newDist = 4
NOT Updated : current = a neighbor = c newDist = 1
Updated : current = c neighbor = b newDist = 3
Updated : current = c neighbor = d newDist = 5
Updated : current = b neighbor = e newDist = 7
NOT Updated : current = d neighbor = e newDist = 7


In [26]:
for v in G.verDictionary.values():
    print(source.getVertexID(), " to ", v.getVertexID(), "-->", v.getDistance())

path = [destination.getVertexID()]
shortest(destination, path)
print ('The shortest path from a to e is: %s' % (path[::-1]))

a  to  a --> 0
a  to  b --> 3
a  to  c --> 1
a  to  d --> 5
a  to  e --> 7
The shortest path from a to e is: ['a', 'c', 'b', 'e']


### <a id='Ex1'>Ex.1 The Maze</a>

There is a ball in a maze with empty spaces and walls. The ball can go through empty spaces by rolling up, down, left or right.

Given the ball's start position, the destination and the maze, determine whether the ball could stop at the destination.

The maze is represented by a binary 2D array. 1 means the wall and 0 means the empty space. You may assume that the borders of the maze are all walls. The start and destination coordinates are represented by row and column indexes.


<img src="./images/ch17/maze1.png" width="560"/>

In [33]:
def sol1_dfs(matrix, start, destination):
    visited = [[False]*len(matrix[0]) for i in range(len(matrix))]
    return dfs_helper(matrix, start, destination, visited)

def dfs_helper(matrix, start, destination, visited):
    if start[0] == destination[0] and start[1] == destination[1]:
        return True

    if visited[start[0]][start[1]] is True:
        return False

    if matrix[start[0]][start[1]] == 1:
        return False

    visited[start[0]][start[1]] = True
    print(start,end="->")

    if start[1] < len(matrix[0])-1:
        r = (start[0], start[1]+1)
        if dfs_helper(matrix, r, destination, visited):
            return True

    if start[0] < len(matrix)-1:
        d = (start[0]+1, start[1])
        if dfs_helper(matrix, d, destination, visited):
            return True

    if start[1] > 0:
        l = (start[0], start[1]-1)
        if dfs_helper(matrix, l, destination, visited):
            return True

    if start[0] > 0:
        u = (start[0]-1, start[1])
        if dfs_helper(matrix, u, destination, visited):
            return True

    # move = [(-1, 0), (0, 1), (1, 0), (0, -1)]
    #
    # for i, j in move:
    #
    #     new_row, new_col = start[0] + i, start[1] + j
    #
    #     if not (new_row < 0 or new_row >= len(matrix) or new_col < 0 or new_col >= len(matrix[0])):
    #         start = (new_row,new_col)
    #         if dfs_helper(matrix, start, destination, visited):
    #             return True

    return False

In [34]:
matrix1 = [
    [0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0],
    [1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0]
]

start = (0, 0)
dest  = (4, 4)
sol1_dfs(matrix1, start, dest)

(0, 0)->(0, 1)->(1, 1)->(1, 2)->(1, 3)->(1, 4)->(2, 4)->(0, 4)->(0, 3)->(2, 2)->(2, 1)->(2, 0)->(1, 0)->

False

In [35]:
matrix2 = [
    [0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0],
    [1, 1, 0, 1, 1],
    [0, 0, 0, 0, 0]
]

start = (0, 0)
dest  = (4, 4)
sol1_dfs(matrix2, start, dest)

(0, 0)->(0, 1)->(1, 1)->(1, 2)->(1, 3)->(1, 4)->(2, 4)->(0, 4)->(0, 3)->(2, 2)->(3, 2)->(4, 2)->(4, 3)->

True

In [36]:
def sol1_dfs_iterative(matrix, start, destination):
    if start[0] == destination[0] and start[1] == destination[1]:
        return True
    
    stack = []
    visited = [[False]*len(matrix[0]) for i in range(len(matrix))]
    move = [(0, 1), (1, 0), (0, -1), (-1, 0)]  #→，↓，←, ↑
    stack.append(start)
    
    while len(stack):
        cur = stack.pop()
        
        if cur[0] == destination[0] and cur[1] == destination[1]:
            return True
        
        if visited[cur[0]][cur[1]] is True or matrix[cur[0]][cur[1]]==1:
            continue
            
        visited[cur[0]][cur[1]] = True
        print(cur,end="->")
        for i, j in move:
            new_row, new_col = cur[0] + i, cur[1] + j
            if not (new_row < 0 or new_row >= len(matrix) or new_col < 0 or new_col >= len(matrix[0])):
                stack.append((new_row, new_col))
    
    return False

In [37]:
sol1_dfs_iterative(matrix1, start, dest)

(0, 0)->(1, 0)->(2, 0)->(2, 1)->(1, 1)->(0, 1)->(1, 2)->(2, 2)->(1, 3)->(0, 3)->(0, 4)->(1, 4)->(2, 4)->

False

In [38]:
sol1_dfs_iterative(matrix2, start, dest)

(0, 0)->(1, 0)->(2, 0)->(2, 1)->(1, 1)->(0, 1)->(1, 2)->(2, 2)->(3, 2)->(4, 2)->(4, 1)->(4, 0)->(4, 3)->

True

In [54]:
from collections import deque

def sol1_bfs(matrix, start, destination):
    if start[0] == destination[0] and start[1] == destination[1]:
        return True

    queue = deque()
    visited = [[False] * len(matrix[0]) for i in range(len(matrix))]
    move = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # →，↓，←, ↑
    visited[start[0]][start[1]] = True
    queue.append(start)

    while len(queue):
        cur = queue.popleft()
        if cur[0] == destination[0] and cur[1] == destination[1]:
            return True
        print(cur, end='->')
        for i, j in move:
            x = cur[0] + i
            y = cur[1] + j

            if x < 0 or x >= len(matrix) or y < 0 or y >= len(matrix[0]): # 要先判断边界条件 否则会出错
                continue
                
            if visited[x][y] is True or matrix[x][y] == 1:
                continue

            visited[x][y] = True
            queue.append((x, y))

    return False

In [55]:
sol1_bfs(matrix1, start, dest)

(0, 0)->(0, 1)->(1, 0)->(1, 1)->(2, 0)->(1, 2)->(2, 1)->(1, 3)->(2, 2)->(1, 4)->(0, 3)->(2, 4)->(0, 4)->

False

In [56]:
sol1_bfs(matrix2, start, dest)

(0, 0)->(0, 1)->(1, 0)->(1, 1)->(2, 0)->(1, 2)->(2, 1)->(1, 3)->(2, 2)->(1, 4)->(0, 3)->(3, 2)->(2, 4)->(0, 4)->(4, 2)->(4, 3)->(4, 1)->

True

### <a id='Ex2'>Ex.2 The Maze II</a>

There is a ball in a maze with empty spaces and walls. The ball can go through empty spaces by rolling up, down, left or right. <font color='Red'>but it won't stop rolling until hitting a wall. When the ball stops, it could choose the next direction.</font>

Given the ball's start position, the destination and the maze, determine whether the ball could stop at the destination.

The maze is represented by a binary 2D array. 1 means the wall and 0 means the empty space. You may assume that the borders of the maze are all walls. The start and destination coordinates are represented by row and column indexes.

<img src="./images/ch17/maze2.png" width="640"/>
<img src="./images/ch17/maze3.png" width="640"/>

### <a id='Ex3'>Ex.3 The Maze III</a>

There is a ball in a maze with empty spaces and walls. The ball can go through empty spaces by rolling up, down, left or right, but it won't stop rolling until hitting a wall. When the ball stops, it could choose the next direction.

Given the ball's start position, the destination and the maze, <font color='red'>find the shortest distance for the ball to stop at the destination</font>. The distance is defined by the number of empty spaces traveled by the ball from the start position (excluded) to the destination (included). If the ball cannot stop at the destination, return -1.

The maze is represented by a binary 2D array. 1 means the wall and 0 means the empty space. You may assume that the borders of the maze are all walls. The start and destination coordinates are represented by row and column indexes.

<img src="./images/ch17/maze4.png" width="640"/>
<img src="./images/ch17/maze5.png" width="640"/>

### <a id='Ex4'>Ex.4 The Maze IV</a>

There is a ball in a maze with empty spaces and walls. The ball can go through empty spaces by rolling up (u), down (d), left (l) or right (r), but it won't stop rolling until hitting a wall. When the ball stops, it could choose the next direction. There is also a hole in this maze. <font color="red">The ball will drop into the hole if it rolls on to the hole</font>.

Given the ball position, the hole position and the maze, find out how the ball could drop into the hole by moving the shortest distance. The distance is defined by the number of empty spaces traveled by the ball from the start position (excluded) to the hole (included). Output the moving directions by using 'u', 'd', 'l' and 'r'. Since there could be several different shortest ways, you should output the lexicographically smallest way. If the ball cannot reach the hole, output "impossible".

The maze is represented by a binary 2D array. 1 means the wall and 0 means the empty space. You may assume that the borders of the maze are all walls. The ball and the hole coordinates are represented by row and column indexes.